# <img src ='https://airsblobstorage.blob.core.windows.net/airstream/databricks.png' width="50px"> Mounting ADLS Gen2

[First Create Your Key Vault Secret Scope](https://docs.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes)

[Accessing Azure Data Lake Storage Gen2 and Blob Storage with Azure Databricks](https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/azure-storage)

In [0]:
#FIRST CREATE YOUR KEY VAULT SECRET SCOPE 
#Tutorial: https://docs.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes

#You need to go here to create your secret Scope
#https://YOURDATABRICKSINSTANCE.azuredatabricks.net#secrets/createScope

In [0]:
#THESE ARE UNIQUE TO ME YOU WILL HAVE TO CHANGE THEM FOR YOU 
#https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/azure-storage

storage_account_name = 'YOURDATALAKE'
client_id            = dbutils.secrets.get(scope="akv-bck-scope", key="databricks-app-client-id")
tenant_id            = dbutils.secrets.get(scope="akv-bck-scope", key="databricks-app-tenant-id")
client_secret        = dbutils.secrets.get(scope="akv-bck-scope", key="databricks-app-client-secret")

configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": f"{client_id}",
           "fs.azure.account.oauth2.client.secret": f"{client_secret}",
           "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}


#YOURDATALAKE                - east - LRS
#YOURDATALAKE-GEO-REDUNTANT  - east - RA-GRS
#YOURDATALAKE-WEST           - west - LRS

In [0]:
#Unmount a Mount
dbutils.fs.unmount("/mnt/YOURDATALAKE")

In [0]:
#Now create your mount using a Service Principal
container_name = "managed-flight"
dbutils.fs.mount(
  source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
  mount_point = f"/mnt/{storage_account_name}",
  extra_configs = configs)

In [0]:
%fs ls /mnt/

In [0]:
#Get list of files and folders inside a mount
display(
  dbutils.fs.ls("/mnt/YOURDATALAKE")
)

In [0]:
%fs
ls dbfs:/databricks-datasets/airlines

In [0]:
#load one file to review
df = spark.read.load("/databricks-datasets/airlines/part-00000",format="csv",sep=",",inferSchema="true",header="true" )

#Infer the schema
csv_schema = df.schema

#Read all the files into a dataframe
df = spark \
    .read \
    .format("csv")\
    .schema(csv_schema) \
    .load("/databricks-datasets/airlines/part-*")

In [0]:
df = df.filter(df.Year.isNotNull()).drop()

In [0]:
display(df)

In [0]:
# This will take a few minutes to run since this is the first execution step it will be executing the DAG 
df.count()

In [0]:
%sql
DROP DATABASE IF EXISTS flight_delta

In [0]:
%sql
DROP TABLE flight_delta.FlightDeltaLake

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS flight_delta
LOCATION "/mnt/YOURDATALAKE"  --this is the location of the deltalake, since we are providing a location it is unmanaged